In [6]:
import xlrd
import pandas as pd
import random 
import datetime

def opendrillprog(filename, tabname):
    
    try:
        xls = pd.ExcelFile(filename)
        df = xls.parse(tabname, skiprows=0, index_col=None, na_values=['NA'])
        print('Opening the Drilling Program')
    except:
        print('No Spreadsheet for Drilling Program')
        return
    
    #xls = pd.ExcelFile(filename)
    #df = xls.parse(tabname, skiprows=0, index_col=None, na_values=['NA'])
    #print(df.sample(2))
    startdate = df['DateStamp'].min()
    enddate = df['DateStamp'].max()
    months = len(df)
    years = months/12
    #print(startdate, enddate)
# Get some stats
    TotalCap = 0
    TotalLand = 0
    Numwells = df['Well Count'].sum() 
    TotalDrillCap = df['Drilling Capital'].sum() 
    TotalCompCap = df['Completion Cost ($M)'].sum() 
    TotalConstCap = df['Construction Cost'].sum() 
    TotalLandCap = df['Land costs ($mm)'].sum() 
    TotalTieCap = df['Tie in Costs ($M)'].sum()
    print('Drilling Program Read')
    #print(df)
    #global df
    return df

def readhistdata(filename, tabname):
    try:
        xls = pd.ExcelFile(filename)
        df = xls.parse(tabname, skiprows=0, index_col=None, na_values=['NA'])
    except:
        print('No Spreadsheet for Historical Data')
        return
    try:
        xls = pd.read_csv("Prod.csv")
        df = xls.parse('Historical Data', skiprows=0, index_col=None, na_values=['NA'])
    except:
        print('No CSV for Historical Data')
        print('No History File')
        return

filename = 'Drilling_Program.xlsx'
tabname = 'Drilling Program'
opendrillprog(filename, tabname)
#print(df)
filename = 'readhistdata.xlsx'
tabname = 'Historical Data'
readhistdata(filename, tabname)

#Set some parameters
#''''''''''''''''''''''''''''''''''''''''''''''''
#' Move to the sheet that contains the information for constraining the realizations
#''''''''''''''''''''''''''''''''''''''''''''''''

startdate = datetime.datetime(2009, 10, 5)
enddate = datetime.datetime(2039, 10, 5)
OutputFileName = 'MCF_Out.csv'
#Cells(11, 2).Value = "Include Historic Data? (1=Yes, 0=No)"
Formation = 'Viking'
CaseName = 'Viking Test'
 
# Any number can be used in place of '0'. 
random.seed(97097)
rand = (random.randint(0,1))   
#'Number of Realizations'
numreal = 10

# "Uncertainty Factors Affecting Production"
#"Prob of Geology"
cog = 0.95
#"Prob of Completion % (skin)"
drillrisk = 1.0

# "Uncertainty Factors Affecting Capital"
# "Uncertainty on Drill Cap ((0=none, Lognorm: mean, var))"
# DOnt forget there is a test later to determine if this is used. Same next vars
DrillCapvar = (0,0)
#"Uncertainty on Compl Cap (0=none, Lognorm: mean, var))"
ComplCapvar = (0,0)
# Uncertainty on Const Cap (0=none, Lognorm: mean, var))"
ConstCapvar = (0,0)
#Uncertainty on Land Cap ((0=none, Lognorm: mean, var))"
LandCapvar = (0,0)
#Uncertainty on Tin in Cap ((0=none, Lognorm: mean, var))"
TieCapvar = (0,0)

#Learning Curve (0=none, effect, min capital, max capital)"
learn = 0
capmin = 0
capmax = 9000000000
Capital = -999
capvar = 0

peakrate = 15000

VarWI = 0

Years = enddate.year - startdate.year
Months =  Years * 12 + (enddate.month - startdate.month)   

#If Years < 0 Then MsgBox ("Start and End Dates Reversed")
#'Months = DateDiff("m", startdate, enddate)
intYear = startdate.year
intMonth = startdate.month
intDay = 1
Count = 31

# now build the outprod dataframe
#datestamp = pd.date_range(start = startdate, end = enddate, freq = 'M')
#outdf = []
#outdf['datestamp'] = pd.date_range(start = startdate, end = enddate, freq = 'M')
index = pd.date_range(startdate, periods=Months, freq='M')
 
columns = ['realization','oil','water', 'gas','Oil not process','Processed Water','oil crown','GAS CROWN'
           ,'Stored Oil','Stored Water','Total Oil Production','total water prod','newmaxoil']
 
outdf = pd.DataFrame(index=index, columns=columns)
outdf = outdf.fillna(0)
outdf['realization'] = int(1)
#print(outdf) 
duster = 1

Opening the Drilling Program
Drilling Program Read
No Spreadsheet for Historical Data


In [ ]:


For j = 1 To Years + 1
   For i = 1 To 12
      dtmNewDate = DateSerial(intYear, intMonth, intDay)
      Cells(Count, 1).Value = dtmNewDate
      intMonth = intMonth + 1
      Count = Count + 1
   Next i
   intMonth = 1
   intYear = intYear + 1
Next j

duster = 1

''''''''''''''''''''''''''''''''''''''''''''''''
'Arrays
''''''''''''''''''''''''''''''''''''''''''''''''

Dim Outprod() As Single

ReDim Outprod(1 To 15, 1 To Months, 1 To Realizations)

' Outprod array variables

'8888888888888888888888888888888888888888888888888888888888888888888888888888888
'8888888888888888888888888888888888888888888888888888
'1     2       3    4               5                6          7            8             9             10                    11               12
'oil   water gas    Oil not process Processed Water  oil crown  GAS CROWN    Stored Oil    
Stored Water  Total Oil Production  total water prod newmaxoil

'Now layer in the historical data

adddata = 1

If adddata = 1 Then
'For n = 1 To HistMonths - Pastedate
'   Outprod(1, n, r) = HistoricData(Pastedate + n, 9)
'   Next n
'   End If
            
 HistDate = PasteDate - 1
 For i = 1 To Realizations
    HistDate = PasteDate - 1
    For j = 1 To Months
       If HistDate < HistMonths Then
          histprod = HistoricData(HistDate, 4)
          histwat = HistoricData(HistDate, 5)
          histprod = histprod * adddata
       Else
          histprod = 0
       End If
       Outprod(1, j, i) = histprod
       Outprod(2, j, i) = histwat
       Outprod(10, j, i) = histprod
       Outprod(11, j, i) = histwat
       HistDate = HistDate + 1
       Next j
       Next i

End If
       
porsche = 1
       
       
Dim Outcap() As Single

ReDim Outcap(1 To 5, 1 To Months, 1 To Realizations)

Dim wellcount() As Single

ReDim wellcount(1 To 5, 1 To Months, 1 To Realizations)

Dim Results() As Single

ReDim Results(1 To 300, 1 To Months, 1 To Realizations)

Dim Sumstats() As Single
ReDim Sumstats(1 To 20, 1 To Months, 1 To Realizations)

Dim Temp() As Single
ReDim Temp(1 To Realizations)

Dim TVM() As Single
ReDim TVM(1 To 25, 1 To Years, 1 To Realizations)

Dim capstats() As Single
ReDim capstats(1 To Numwells, 1 To Realizations)


'    Range("a16:iv500").Select
'    Selection.ClearContents

'''''''''''''''''''''''''''''''''''''''
'Set column names
'''''''''''''''''''''''''''''''''''''''
 
K = 3

'Set counter for location of output arrays
l = 1

'Start Generating Realizations
 
 For r = 1 To Realizations
    T = 1
    numprod = 0
    drilledwells = 0

rignum = 0
If rignum = 0 Then

' Set well count to 0

drilledwells = 0

'Start drill program

For q = 1 To Months

